In [1]:
import os
import sys

import pandas as pd

module_path = os.path.abspath(os.path.join(".."))
sys.path.insert(0, module_path)


from planning.config import (  # noqa: E402
    employee_restrictions,
)
from planning.employee import (  # noqa: E402
    create_employees_with_dates,
    generate_summary_total,
    generate_transposed_excel_with_styles,
    init_employees_by_shifts,
    load_employees_from_yaml,
)

year = 2025
start_date = f"{year}-01-01"

employees_file = os.path.join("..", "..", "data", "2025", "case_1", "employees.yaml")
planning_file = os.path.join("..", "..", "data", "2025", "case_1", "data.xlsx")

employees = load_employees_from_yaml(employees_file, employee_restrictions)
employees_info, dates = create_employees_with_dates(start_date, 365, employees)
all_employees_by_shift = init_employees_by_shifts(dates, employee_restrictions)


In [2]:
planning_data = pd.read_excel(planning_file, index_col=0, na_filter=False, sheet_name=None)

planning_data

{'01':    1/1/25 2/1/25 3/1/25 4/1/25 5/1/25 6/1/25 7/1/25 8/1/25 9/1/25 10/1/25  \
                                                                             
         X      J      V      S      D      L      M      X      J       V   
 E1             M      M      M      M             T      T      T           
 E2             M      M      M      M             T      T      T           
 E3      M             T      T      T      T                    M       M   
 E4      M             T      T      T      T                    M       M   
 E5      T      T                           M      M      M              T   
 E6      T      T                           M      M      M              T   
 
     ... 22/1/25 23/1/25 24/1/25 25/1/25 26/1/25 27/1/25 28/1/25 29/1/25  \
     ...                                                                   
     ...       X       J       V       S       D       L       M       X   
 E1  ...               M       M       M       M              

In [3]:
total_data = generate_summary_total(employees, employee_restrictions, planning_data)

In [4]:
total_data

,THT,MHA,Diff
,,,
E1,1837.5,1852.5,15.0
E2,1837.5,1852.5,15.0
E3,1800.0,1852.5,52.5
E4,1800.0,1852.5,52.5
E5,1822.5,1852.5,30.0
E6,1822.5,1852.5,30.0


In [5]:
planning_data["12"]

,1/12/25,2/12/25,3/12/25,4/12/25,5/12/25,6/12/25,7/12/25,8/12/25,9/12/25,10/12/25,...,23/12/25,24/12/25,25/12/25,26/12/25,27/12/25,28/12/25,29/12/25,30/12/25,31/12/25,THT
,,,,,,,,,,,,,,,,,,,,,
,L,M,X,J,V,S,D,L,M,X,...,M,X,J,V,S,D,L,M,X,NaN
E1,T,,,M,M,M,M,,T,T,...,,,M,M,M,M,,T,T,157.5
E2,T,,,M,M,M,M,,T,T,...,,,M,M,M,M,,T,T,157.5
E3,M,M,M,,T,T,T,T,,,...,M,M,,T,T,T,T,,,157.5
E4,M,M,M,,T,T,T,T,,,...,M,M,,T,T,T,T,,,157.5
E5,,T,T,T,,,,M,M,M,...,T,T,T,,,,M,M,M,150.0
E6,,T,T,T,,,,M,M,M,...,T,T,T,,,,M,M,M,150.0


In [9]:
generate_transposed_excel_with_styles(planning_data["01"], employee_restrictions, f"samples/{year}_planning_0.xlsx")
with pd.ExcelWriter(f"samples/{year}_planning_0.xlsx", engine='xlsxwriter') as writer:
    startrow = 0
    for month, data in planning_data.items():
        data.to_excel(writer, sheet_name='Planning', startrow=startrow)
        startrow += data.shape[0] + 2  # Add 2 rows as a separator
    total_data.to_excel(writer, sheet_name='Total', startrow=0)